In [ ]:
import sys
sys.path.append("../")
from clapton.clapton import claptonize, claptonize_opt
from clapton.ansatzes import circular_ansatz
from clapton.utils import Results
import numpy as np
import pickle
import time
import copy


from qiskit_nature.second_quantization.drivers import PySCFDriver
from qiskit_nature.second_quantization.problems import ElectronicStructureProblem
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.mappers.second_quantization import JordanWignerMapper
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit.algorithms import NumPyMinimumEigensolver


d:\research-impl\clapton-cafqa-opt\clapton\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

# 1. Set up the molecule using PySCF driver via Qiskit
driver = PySCFDriver(atom='H 0 0 0; H 0 0 0.735', basis='sto3g')
es_problem = ElectronicStructureProblem(driver)

# 2. Build second-quantized Hamiltonian
second_q_ops = es_problem.second_q_ops()
electronic_hamiltonian = second_q_ops[0]  # The first operator is the Hamiltonian

# 3. Map to qubit Hamiltonian using Jordan-Wigner
qubit_converter = QubitConverter(mapper=JordanWignerMapper())
qubit_op = qubit_converter.convert(electronic_hamiltonian, num_particles=es_problem.num_particles)

# 4. Print Pauli terms and coefficients
for pauli_term in qubit_op.to_list():
    print(pauli_term)


In [ ]:
GA_results = Results("GA")
optuna_results = Results("optuna")


In [ ]:
# define Hamiltonian, e.g. 3q Heisenberg model with random coefficients
paulis_ = ["XXI", "IXX", "YYI", "IYY", "ZZI", "IZZ"]
np.random.seed(42)  # You can replace 42 with any integer seed
coeffs_ = np.random.random(len(paulis_))

vqe_pcirc_ = circular_ansatz(N=len(paulis_[0]), reps=1, fix_2q=True)


vqe_pcirc = copy.deepcopy(vqe_pcirc_)
paulis = copy.deepcopy(paulis_)
coeffs = copy.deepcopy(coeffs_)

GA_time = time.time()
ks_best, _, energy_best = claptonize(
    paulis,
    coeffs,
    vqe_pcirc,
    n_proc=4,           # total number of processes in parallel
    n_starts=4,         # number of random genetic algorithm starts in parallel
    n_rounds=1,         # number of budget rounds, if None it will terminate itself
    # callback=print,     # callback for internal parameter (#iteration, energies, ks) processing
    budget=20           # budget per genetic algorithm instance
)
GA_time = time.time() - GA_time

GA_results.add_result("-", paulis, coeffs.tolist(), GA_time, ks_best, float(energy_best))

vqe_pcirc = copy.deepcopy(vqe_pcirc_)
paulis = copy.deepcopy(paulis_)
coeffs = copy.deepcopy(coeffs_)

optuna_time = time.time()
ks_best, _, energy_best = claptonize_opt(
    paulis,
    coeffs,
    vqe_pcirc,
)
optuna_time = time.time() - optuna_time
optuna_results.add_result("-", paulis, coeffs.tolist(), optuna_time, ks_best, float(energy_best))

with open("results/GA", "wb") as file:
    pickle.dump(GA_results, file)
with open("results/optuna", "wb") as file:
    pickle.dump(optuna_results, file)

d:\research-impl\clapton-cafqa-opt\clapton\venv\Lib\site-packages\pygad\pygad.py:1139: UserWarning: The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.
  warnings.warn("The 'delay_after_gen' parameter is deprecated starting from PyGAD 3.3.0. To delay or pause the evolution after each generation, assign a callback function/method to the 'on_generation' parameter to adds some time delay.")


STARTING ROUND 0


started GA at id None with 1 procs

